## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
!pip install -q torch==2.0.0 transformers==4.28.1 watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
# Imports
import torch
import sklearn
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [4]:
# Mostra somente mensagens de erro
from transformers import logging
logging.set_verbosity_error()

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "version 0" --iversions

Author: version 0

torch       : 2.0.0
transformers: 4.28.1
sklearn     : 1.2.2
numpy       : 1.25.2



## Construção da Classe de Tokenização dos Dados

Conversão de texto em representação números

In [6]:
class TokenizaDados(Dataset):
    """
    Classe para tokenização dos dados.

    Args:
        texts (list): Lista de textos a serem tokenizados.
        labels (list): Lista de rótulos associados aos textos.
        tokenizer: Objeto tokenizer para realizar a tokenização.
        max_length (int): Comprimento máximo para o token dos textos.

    Attributes:
        texts (list): Lista de textos a serem tokenizados.
        labels (list): Lista de rótulos associados aos textos.
        tokenizer: Objeto tokenizer para realizar a tokenização.
        max_length (int): Comprimento máximo para o token dos textos.
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Método construtor da classe TokenizaDados.

        Args:
            texts (list): Lista de textos a serem tokenizados.
            labels (list): Lista de rótulos associados aos textos.
            tokenizer: Objeto tokenizer para realizar a tokenização.
            max_length (int): Comprimento máximo para o token dos textos.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Retorna o número de textos na lista.

        Returns:
            int: Número de textos na lista.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Retorna um item tokenizado específico.

        Args:
            idx (int): Índice do texto na lista.

        Returns:
            dict: Dicionário contendo os IDs dos tokens, máscara de atenção e rótulo.
        """
        # Obtém o índice do texto e do label
        text = self.texts[idx]
        label = self.labels[idx]

        # Aplica a tokenização
        inputs = self.tokenizer.encode_plus(text,
                                            add_special_tokens=True,
                                            max_length=self.max_length,
                                            padding='max_length',
                                            truncation=True,
                                            return_tensors='pt')

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label)
        }

## Funções Para os Loops de Treino, Avaliação e Previsão com Novos Dados

In [7]:
def treina_modelo(model, data_loader, criterion, optimizer, device):
    """
    Treina um modelo de aprendizado de máquina em um conjunto de dados.

    Parâmetros:
    model (torch.nn.Module): O modelo a ser treinado.
    data_loader (torch.utils.data.DataLoader): O DataLoader contendo o conjunto de dados.
    criterion: A função de perda a ser usada para calcular o erro.
    optimizer: O otimizador utilizado para atualizar os pesos do modelo.
    device (torch.device): O dispositivo onde os cálculos serão realizados (CPU ou GPU).

    Retorna:
    float: A média do erro de treinamento durante a época.
    """

    # Coloca o modelo em modo de treino
    model.train()

    # Inicializa o erro com zero
    total_loss = 0

    # Loop pelo data loader
    for batch in data_loader:

        # Extrai os ids do batch de dados e coloca no device
        input_ids = batch['input_ids'].to(device)

        # Extrai a máscara e coloca no device
        attention_mask = batch['attention_mask'].to(device)

        # Extrai os labels e coloca no device
        labels = batch['label'].to(device)

        # Zera os gradientes
        optimizer.zero_grad()

        # Faz as previsões
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        # Extrai o erro do modelo
        loss = outputs.loss

        # Aplica a otimização com backpropagation
        loss.backward()
        optimizer.step()

        # Acumula o erro
        total_loss += loss.item()

    # Retorna a média do erro de treinamento
    return total_loss / len(data_loader)


In [8]:
def avalia_modelo(model, data_loader, criterion, device):
    """
    Avalia um modelo usando um data_loader e um critério de avaliação.

    Parâmetros:
    -----------
    model : torch.nn.Module
        O modelo a ser avaliado.

    data_loader : torch.utils.data.DataLoader
        DataLoader contendo os dados de avaliação.

    criterion : torch.nn.Module
        Critério de avaliação (como uma função de perda).

    device : torch.device
        Dispositivo de computação (CPU ou GPU) onde o modelo será executado.

    Retorna:
    --------
    float
        Média da perda calculada sobre todo o data_loader.
    """

    # Define o modelo no modo de avaliação
    model.eval()

    # Inicializa a variável para armazenar a soma total da perda
    total_loss = 0

    # Desativa os gradientes durante a avaliação
    with torch.no_grad():

        # Itera sobre os batches no data_loader
        for batch in data_loader:

            # Move os dados para o dispositivo especificado
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Calcula as saídas do modelo
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            # Obtém a perda do output
            loss = outputs.loss

            # Acumula a perda total
            total_loss += loss.item()

    # Retorna a média da perda
    return total_loss / len(data_loader)


In [9]:
def predict(model, data_loader, device):
    """
    Realiza previsões usando um modelo treinado.

    Args:
    - model (torch.nn.Module): O modelo treinado para fazer previsões.
    - data_loader (torch.utils.data.DataLoader): Um DataLoader contendo os dados a serem usados para previsão.
    - device (torch.device): O dispositivo (CPU ou GPU) onde o modelo será executado.

    Returns:
    - predictions (list): Uma lista contendo as previsões para cada lote de dados no DataLoader.

    Exemplo:
    >>> model = MyModel()
    >>> data_loader = DataLoader(dataset)
    >>> device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    >>> predictions = predict(model, data_loader, device)
    """

    # Coloca o modelo em modo de avaliação
    model.eval()

    # Lista para armazenar as previsões
    predictions = []

    # Desabilita o cálculo dos gradientes durante a avaliação
    with torch.no_grad():

        # Itera sobre cada lote de dados no DataLoader
        for batch in data_loader:

            # Move os dados para o dispositivo
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Faz a previsão usando o modelo
            outputs = model(input_ids, attention_mask=attention_mask)

            # Obtém as previsões finais (classe com maior probabilidade)
            _, preds = torch.max(outputs.logits, dim=1)

            # Extende a lista de previsões com as previsões atuais
            predictions.extend(preds.tolist())

    return predictions


## Definição dos Dados

In [10]:
# Hiperparâmetros
EPOCHS = 10
BATCH_SIZE = 16
MAX_LENGTH = 64
LEARNING_RATE = 2e-5
RANDOM_SEED = 42

In [11]:
# Conjunto de dados de exemplo
texts = [
    'A velocidade da luz é aproximadamente 300.000 km/s.',
    'A Terra é plana e os répteis controlam o mundo.',
    'A fotossíntese é um processo importante para as plantas.',
    'As vacas podem voar e atravessar paredes de concreto.',
    'O oxigênio é essencial para a respiração dos seres vivos.',
    'Os cavalos podem falar como seres humanos.',
    'As crianças aprendem a partir dos exemplos dos pais.',
    'As palavras verdadeiras não são agradáveis e as agradáveis não são verdadeiras.',
    'Leopardos trabalham de terno e gravata em frente ao computador.',
    'Carros voadores estão por toda parte.'
]

In [12]:
labels = [0, 1, 0, 1, 0, 1, 0, 0, 1, 1]  # 0: normal, 1: anômala

In [13]:
# Divisão dos dados em treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(texts,
                                                                      labels,
                                                                      test_size = 0.2,
                                                                      random_state = RANDOM_SEED)

## Tokenização dos Dados e Criação dos DataLoaders

In [14]:
# Nome do modelo pré-treinado com 110M de parâmetros
PRETRAINED_MODEL = 'bert-base-uncased'

https://huggingface.co/bert-base-uncased

In [15]:
# Inicializa o tokenizador
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [16]:
# Tokenização dos dados
train_dataset = TokenizaDados(train_texts, train_labels, tokenizer, MAX_LENGTH)
test_dataset = TokenizaDados(test_texts, test_labels, tokenizer, MAX_LENGTH)

In [17]:
# Data Loaders
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [18]:
# Configuração do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Construção, Treinamento e Avaliação do Modelo

In [19]:
# Importa o modelo pré-treinado
modelo = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels = 2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [20]:
# Coloca o modelo na memória do device
modelo.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
# Configuração do otimizador e critério de perda
optimizer = torch.optim.AdamW(modelo.parameters(), lr = LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [22]:
# Treinamento e validação do modelo
for epoch in range(EPOCHS):

    train_loss = treina_modelo(modelo, train_loader, criterion, optimizer, device)

    test_loss = avalia_modelo(modelo, test_loader, criterion, device)

    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss}, Test Loss: {test_loss}')

Epoch 1/10, Train Loss: 0.6387425661087036, Test Loss: 1.1075495481491089
Epoch 2/10, Train Loss: 0.5123462677001953, Test Loss: 1.1044825315475464
Epoch 3/10, Train Loss: 0.40835487842559814, Test Loss: 1.2408448457717896
Epoch 4/10, Train Loss: 0.4219149053096771, Test Loss: 1.2373809814453125
Epoch 5/10, Train Loss: 0.3286563754081726, Test Loss: 1.210616111755371
Epoch 6/10, Train Loss: 0.2878265976905823, Test Loss: 1.2965350151062012
Epoch 7/10, Train Loss: 0.2285674512386322, Test Loss: 1.3309810161590576
Epoch 8/10, Train Loss: 0.2041279375553131, Test Loss: 1.4575189352035522
Epoch 9/10, Train Loss: 0.21173195540905, Test Loss: 1.5781400203704834
Epoch 10/10, Train Loss: 0.17372822761535645, Test Loss: 1.7895417213439941


## Deploy e Uso do Modelo Treinado

In [23]:
# Teste de detecção de anomalias
novos_dados = ['A gravidade mantém os planetas em órbita ao redor do Sol.',
               'Os carros podem nadar no oceano como peixes.']

In [24]:
# Tokeniza a amostra de dados
novo_dataset = TokenizaDados(novos_dados, [0] * len(novos_dados), tokenizer, MAX_LENGTH)

In [25]:
# Cria o dataloader
novo_loader = DataLoader(novo_dataset, batch_size = BATCH_SIZE)

In [26]:
# Faz as previsões com o modelo
previsoes = predict(modelo, novo_loader, device)

In [27]:
for text, prediction in zip(novos_dados, previsoes):
    print(f'Sentença: {text} | Previsão: {"anômala" if prediction else "normal"}')

Sentença: A gravidade mantém os planetas em órbita ao redor do Sol. | Previsão: normal
Sentença: Os carros podem nadar no oceano como peixes. | Previsão: anômala
